# 3.A. How we handle numerical data in the ecosystem

In many previous examples it was often the case that the returned result of a function was a `Dataset`. For example `trajectory` and `poincaresos` return a `Dataset`.

What is a `Dataset`?

In [ ]:
using DynamicalSystems, Plots
dataset = Dataset(rand(1000, 3));

A `Dataset` is how we handle numerical data in **DynamicalSystems.jl**. In essense it is a wrapper of `Vector{SVector}` (staticallly sized vectors). Besides that, `Dataset` has matrix-like indexing support:

* when accessing a `Dataset` with a single index it acts as a vector of static vectors

In [ ]:
typeof( dataset[1])

In [ ]:
x = 0.0
for point in dataset # point is a static vector
    x += point[1] - point[2]
end
x

* when accessed with two indices a `Dataset` acts _like_ a matrix where each _row_ is one data point. Equivalently, it is accessed as the matrix it is printed like.

In [ ]:
dataset[1, 2]

In [ ]:
dataset[1, 2] == dataset[1][2]

In [ ]:
dataset[:, 1]

In [ ]:
dataset[1:10, :]

# 3.B. Delay Coordinates Embedding
Let's say you have a "real-world system" which you measure in an experiment or so. You are assuming that the system is composed of several dynamic variables, but you can only measure one of them (or some function of the variable).

You have a severe lack of recorded information for the system. What do you do?
1. Give up on science, it is a complete waste of time.
2. Use [Takens theorem](https://en.wikipedia.org/wiki/Takens%27s_theorem), which is indistinguishable from magic.

**DynamicalSystems.jl** suggests the latter.

From a timeseries $s$ one can *reconstruct* a state-space $\mathbf{z}$ simply by shifting $s$ in time, like
  
  $$\mathbf{z}(n) = (s(n), s(n+\tau), s(n+2\tau), \dots, s(n+\gamma\tau))$$
  
This is done with the `reconstruct(s, γ, τ)` function or the `embed(s, D, τ)` function (with the embedding dimension `D = γ + 1`).

In [ ]:
s = rand(100000)
γ = 3 # how many temporal neighbors?
τ = 1 # delay time
R = reconstruct(s, γ, τ)

`reconstruct` and `embed` return a `Dataset` as well. Because of this they are very fast to compute!

## But how is this even helpful?

What we have done is more than just a fancy way to produce a higher dimensional dataset out of a univariate timeseries. For a proper choice of `τ, γ` [Takens theorem](https://en.wikipedia.org/wiki/Takens%27s_theorem) says that the reconstructed trajectory is homeomorphic with a true, real trajectory of that dynamical system that we got `s` from.

In sort, many quantities like e.g. the Lyapunov exponents, the dimension of the attracting set, etc., are the same for the real dynamical system and the one we reconstructed here numerically. **Even though we are clueless of how many variables the system may have in reality**.

We will be confirming this concept numerically in the next notebook.


## How does a reconstruction look?
The `gissinger` system is a 3D chaotic continuous system

In [ ]:
ds = Systems.gissinger(ones(3))

In [ ]:
using DynamicalSystemsBase

In [ ]:
dt = 0.05
data = trajectory(ds, 2000.0, dt = dt, Ttr=10)

xyz = columns(data)

plot(xyz..., leg=false, title="Gissinger attractor", color=:black)

In [ ]:
typeof(columns(data))

Here are some some examples of reconstructions of the `gissinger` system, using each of the variables of the system, different delay times and embedding dimension of `2` 

In [ ]:
ds = Systems.gissinger(ones(3)) # 3D continuous chaotic system, also shown in orbit diagrams tutorial
dt = 0.05
data = trajectory(ds, 2000.0, dt = dt)


k = 1
subplots = []
for i in 1:3, j in 1:3
    τ =  [5, 30, 100][j]
    R = reconstruct(xyz[i], 1, τ)
    push!(subplots, plot(R[:, 1], R[:, 2], color = :black, lw = 0.8, title="var = $i, \\tau = $τ"))
    k+=1
end

subplots = reshape(subplots, 3,3)
plot(subplots..., layout=(3,3), size=(1000, 1000), legend=false)

Someone familiar with delay coordinates embadding, will see that `τ = 5` is too small while `τ = 100` is too big. `τ = 30` seems okay, but it is not possible to say if it is optimal.

# 3.C. Estimating an optimal delay time

It is important to understand that for Takens theorem to work one still has to choose "appropriately good" values for both the delay time as well as the embedding dimension!

Thankfully, **DynamicalSystems.jl** has full support for that as well!

* `estimate_delay` estimates delay time `τ` using the autocorrelation of the signal
* `estimate_dimension` returns an estimator for the amount of temporal neighbors `γ` using Cao's method

Let's focus on `estimate_delay` for now:

In [ ]:
?estimate_delay

---
For the `gissinger` system above, by looking at the reconstruction plots it seems that a delay time close `30` is a good value.

Let's use some methods for `estimate_delay` and see what we get.

In [ ]:
s = data[:, 1]

methods = ["ac_zero", "mi_min", "exp_decay"]
for method in methods
    τ = estimate_delay(s, method, 0:1:400)
    println("For method = $(method), τ = $τ")
end

What we observe is that "zero of autocorrelation" method failed completely. Also, the "correlation exponential decay" method returned a very bad result. These methods are useful only in specific cases: the first one when the autocorrelation function is oscillatory, and the latter when the correlation decays as an exponential.

On the other side, the first minimum of the mutual information is generally helpful (but also most costly to compute).

---

Let's plot the autocorrelation function and the mutual information 

In [ ]:
using StatsBase

τ = 0:400
ac = autocor(s, τ)
mi = mutualinformation(s, τ)
plot(τ, mi ./ maximum(mi), label = "mutual information")
plot!(τ, ac, label = "autocorrelation")

The autocorrelation not only never crosses zero, but also doesn't decay as an exponential. On the other side, the first minimum of the mutual information of `s` with itself has a value very close to what made sense from plotting the reconstructions.

# 3.D. Fast (self-)Mutual Information

A good method for estimating a proper delay time is the mutual information between a timeseries `s` and itself shifted in time. The function `mutualinformation` computes this quantity. Internally it uses an advanced (to-be-published) algorithm that improves existing standards and can reach high speeds. For example

In [ ]:
@time mutualinformation(s, 1:100)
println("For length: ", length(s))